

```
# Image Classification with Cats vs Dogs dataset
# HPE DSI 312 – Introduction to Deep Learning – Spring 2025
# Project 1

# Load and Prepare Dataset
# The cats_vs_dogs dataset has 25,000 high resolution images and will need to work with a 0,1 format.
```



In [1]:
import itertools
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

IMG_SIZE = 160  # All images will be resized to 160x160 for high resolution pictures.

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs', #specified the datset from tfds that gets loaded in
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'], #Training/Validation/Test
    with_info=True,
    as_supervised=True,
)

# Preprocess for resize and normalize
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image /= 255.0
    return image, label

BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE #automatically tunes

# Applied the preprocessing to my batches
# Added .cache() as it helped with memory and run times
train_batches = raw_train.map(preprocess, num_parallel_calls=AUTOTUNE).cache().shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
validation_batches = raw_validation.map(preprocess, num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_batches = raw_test.map(preprocess, num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

#  Model setup
# Used transfer learning with base model to handle this type of dataset and help with performance.
# Switched from the EfficientNetB0 model to MobileNetV2 for faster processing as the previous model took 10 minutes per epoch.
# Noticed it still had a high accuracy rate at a fraction of the time.
# Classification head to pair with the model and the data's layers.
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the Model
# Used 3 epochs after adjusting the parameters for higher accuracy
EPOCHS = 3
history = model.fit(train_batches,
                    validation_data=validation_batches,
                    epochs=EPOCHS)

loss, accuracy = model.evaluate(test_batches)
print(f"Test Accuracy: {accuracy:.2f}")

Epoch 1/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 507s 855ms/step - accuracy: 0.9234 - loss: 0.1701 - val_accuracy: 0.9776 - val_loss: 0.0662
Epoch 2/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 470s 805ms/step - accuracy: 0.9777 - loss: 0.0594 - val_accuracy: 0.9785 - val_loss: 0.0590
Epoch 3/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 426s 732ms/step - accuracy: 0.9816 - loss: 0.0528 - val_accuracy: 0.9785 - val_loss: 0.0580
73/73 ━━━━━━━━━━━━━━━━━━━━ 51s 697ms/step - accuracy: 0.9788 - loss: 0.0588
Test Accuracy: 0.98


#  Model setup
Used transfer learning with base model to handle this type of dataset and help with performance. Switched from the EfficientNetB0 model to MobileNetV2 for faster processing as the previous model took 10 minutes per epoch. Noticed it still had a high accuracy rate at a fraction of the time. Classification head to pair with the model and the data's layers.
# Compile the Model
Used the adam optimizer as it doesn't require much tuning, loss was for binary responses, and accuracy helps for evaluation.

# Evaluate the Model
With 98 percent accuracy, I chose just accuracy to keep things simple, but recall and and precision could help for more details.A confusion matrix could be a great visual for this binary response.

# Intended uses and potential limitations:
This model can classify cat vs. dog images with decent accuracy of .98 percent. It was a decent sized dataset with limited epoch, but potential for overfitting unless base_model is fine-tuned.

# Improvements
Could invololve adding more data augmentation like flips or rotations, fine tuning top layers, and use a more diverse dataset. I don't think the model is overfitting as validation and training accuracy is so close, if trained longer or used a different base, it could lead to overfitting as stated before.